# Projet NLP : NER avec CamemBERT (Transformers)

## Reconnaissance des Entités Nommées en français avec Fine-tuning de Transformers

## 1. Introduction

### 1.1 Contexte

Ce notebook explore l'utilisation de **CamemBERT**, un modèle Transformer pré-entraîné sur des corpus français massifs, pour la tâche de Reconnaissance des Entités Nommées (NER).

### 1.2 Approche Transformers vs CRF

**CRF (notebook précédent) :**
- Modèle séquentiel classique basé sur des features manuelles
- Entraînement from scratch sur le corpus
- Rapide, léger, ne nécessite pas de GPU
- Performance limitée par les features extraites

**CamemBERT (ce notebook) :**
- Modèle Transformer pré-entraîné sur 138 GB de texte français
- Comprend le contexte bidirectionnel profond
- Fine-tuning sur notre tâche spécifique
- Meilleure performance attendue (+10-20% F1)
- Nécessite plus de ressources (GPU recommandé)

### 1.3 Dataset

MultiCoNER v2 (français) : 68 types d'entités nommées, format BIO
- Train : 16,548 phrases
- Dev : 857 phrases
- Test : 249,786 phrases

## 2. Installation et imports

In [ ]:
# Installation des dépendances (à exécuter une seule fois)
# !pip install transformers torch datasets seqeval accelerate

## Colab (recommandé)

1. Activer GPU : Runtime → Change runtime type → GPU
2. Uploader `data.zip` (contenant `data/fr_train.conll`, `data/fr_dev.conll`, `data/fr_test.conll`)
3. Exécuter la cellule suivante

In [ ]:
# Détection Colab + téléchargement automatique des données depuis GitHub
from pathlib import Path
import urllib.request

try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

BASE_DIR = Path("/content") if IN_COLAB else Path("..")

if IN_COLAB:
    print("Colab détecté. Téléchargement des données depuis GitHub...")
    
    # Créer le dossier data
    data_dir = BASE_DIR / "data"
    data_dir.mkdir(parents=True, exist_ok=True)
    
    # URLs GitHub des fichiers CoNLL (raw content)
    github_raw_url = "https://raw.githubusercontent.com/Badji-M/NLP/main/data"
    files_to_download = ["fr_train.conll", "fr_dev.conll", "fr_test.conll"]
    
    for filename in files_to_download:
        url = f"{github_raw_url}/{filename}"
        filepath = data_dir / filename
        print(f"Téléchargement {filename}...", end=" ")
        try:
            urllib.request.urlretrieve(url, filepath)
            print(f"✓ ({filepath.stat().st_size / (1024*1024):.1f} MB)")
        except Exception as e:
            print(f"✗ Erreur : {e}")
    
    print("\nDonnées téléchargées dans", data_dir)

print("BASE_DIR =", BASE_DIR)

In [ ]:
# Imports système et manipulation de données
import os
import json
import time
from pathlib import Path
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Transformers et PyTorch
import torch
from transformers import (
    CamembertForTokenClassification,
    CamembertTokenizerFast,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
from datasets import Dataset, DatasetDict

# Métriques
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score

# Configuration
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Vérification GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device utilisé : {device}")
if torch.cuda.is_available():
    print(f"GPU : {torch.cuda.get_device_name(0)}")
else:
    print("Attention : CPU détecté. L'entraînement sera lent.")

## 3. Chargement des données

### 3.1 Configuration des chemins

In [ ]:
# DATA_DIR = Path("..") / "data"
DATA_DIR = BASE_DIR / "data"
train_path = DATA_DIR / "fr_train.conll"
dev_path   = DATA_DIR / "fr_dev.conll"
test_path  = DATA_DIR / "fr_test.conll"

# Vérification
for p in [train_path, dev_path, test_path]:
    print(f"{p.name:20s} -> {p.exists()}")

### 3.2 Fonction de lecture CoNLL

In [ ]:
def read_conll(path):
    """
    Lit un fichier CoNLL et retourne les phrases et labels.
    
    Format CoNLL : token TAB _ TAB _ TAB label, phrases séparées par lignes vides
    Format réel : elle _ _ O
                  porte _ _ O
                  susan _ _ B-Artist
    
    Returns:
        sentences: List[List[str]] - liste de phrases (liste de tokens)
        labels: List[List[str]] - liste de labels BIO correspondants
    """
    sentences = []
    labels = []
    
    current_sentence = []
    current_labels = []
    
    with open(path, encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            
            # Ignorer les lignes vides et les commentaires
            if not line or line.startswith("#"):
                if current_sentence:
                    sentences.append(current_sentence)
                    labels.append(current_labels)
                    current_sentence = []
                    current_labels = []
                continue
            
            # Extraction token et label (format : token TAB _ TAB _ TAB label)
            parts = line.split()
            if len(parts) >= 4:
                token = parts[0]
                label = parts[3]
                
                current_sentence.append(token)
                current_labels.append(label)
    
    # Dernière phrase si le fichier ne se termine pas par une ligne vide
    if current_sentence:
        sentences.append(current_sentence)
        labels.append(current_labels)
    
    return sentences, labels

### 3.3 Chargement des trois ensembles

In [ ]:
print("Chargement des données...")
train_sentences, train_labels = read_conll(train_path)
dev_sentences, dev_labels = read_conll(dev_path)
test_sentences, test_labels = read_conll(test_path)

print("\nStatistiques :")
print(f"Train : {len(train_sentences):6,} phrases, {sum(len(s) for s in train_sentences):8,} tokens")
print(f"Dev   : {len(dev_sentences):6,} phrases, {sum(len(s) for s in dev_sentences):8,} tokens")
print(f"Test  : {len(test_sentences):6,} phrases, {sum(len(s) for s in test_sentences):8,} tokens")

print("\nExemple (première phrase train) :")
print(f"Tokens : {train_sentences[0][:10]}...")
print(f"Labels : {train_labels[0][:10]}...")

## 4. Préparation des labels

### 4.1 Construction du mapping label-to-id

In [ ]:
# Récupération de tous les labels uniques
all_labels = set()
for labels in train_labels + dev_labels + test_labels:
    all_labels.update(labels)

# Tri pour reproductibilité
unique_labels = sorted(list(all_labels))

# Création des mappings
label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()}

num_labels = len(unique_labels)

print(f"Nombre de labels uniques : {num_labels}")
print(f"\nPremiers labels : {unique_labels[:10]}")
print(f"Derniers labels : {unique_labels[-10:]}")

# Statistiques sur les entités
entity_labels = [l for l in unique_labels if l.startswith('B-')]
print(f"\nNombre de types d'entités (B-) : {len(entity_labels)}")

### 4.2 Conversion en format Hugging Face Dataset

In [ ]:
def create_dataset(sentences, labels):
    """
    Convertit les données au format Hugging Face Dataset.
    
    Args:
        sentences: List[List[str]]
        labels: List[List[str]]
    
    Returns:
        Dataset avec colonnes 'tokens' et 'ner_tags'
    """
    # Conversion des labels en IDs
    ner_tags = [[label2id[label] for label in sent_labels] for sent_labels in labels]
    
    return Dataset.from_dict({
        "tokens": sentences,
        "ner_tags": ner_tags
    })

# Création des datasets
train_dataset = create_dataset(train_sentences, train_labels)
dev_dataset = create_dataset(dev_sentences, dev_labels)
test_dataset = create_dataset(test_sentences, test_labels)

# Création du DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    "validation": dev_dataset,
    "test": test_dataset
})

print(dataset)
print("\nExemple du dataset train :")
print(train_dataset[0])

## 5. Tokenization avec CamemBERT

### 5.1 Chargement du tokenizer

CamemBERT utilise un tokenizer SentencePiece (subword tokenization) qui découpe les mots en sous-unités. Cela nécessite un alignement spécial avec les labels NER.

In [ ]:
model_checkpoint = "camembert-base"
tokenizer = CamembertTokenizerFast.from_pretrained(model_checkpoint)

print(f"Tokenizer chargé : {model_checkpoint}")
print(f"Vocabulaire : {tokenizer.vocab_size:,} tokens")

# Test du tokenizer
exemple = "La romancière américaine Susan Sontag"
tokens = tokenizer.tokenize(exemple)
print(f"\nExemple de tokenization :")
print(f"Texte  : {exemple}")
print(f"Tokens : {tokens}")

### 5.2 Fonction de tokenization et alignement des labels

Problématique : Un mot peut être découpé en plusieurs subwords. Il faut aligner les labels NER correctement.

Stratégie : 
- Le premier subword d'un mot reçoit le label du mot
- Les subwords suivants reçoivent -100 (ignorés dans le calcul de la loss)

In [ ]:
def tokenize_and_align_labels(examples):
    """
    Tokenize les mots et aligne les labels NER avec les subwords.
    
    Args:
        examples: batch du dataset avec 'tokens' et 'ner_tags'
    
    Returns:
        dict avec input_ids, attention_mask, labels alignés
    """
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding=False  # Le DataCollator gérera le padding
    )
    
    labels = []
    for i, label_ids in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids_aligned = []
        
        previous_word_idx = None
        for word_idx in word_ids:
            # Tokens spéciaux (CLS, SEP, PAD) -> -100
            if word_idx is None:
                label_ids_aligned.append(-100)
            # Début d'un nouveau mot -> label du mot
            elif word_idx != previous_word_idx:
                label_ids_aligned.append(label_ids[word_idx])
            # Continuation d'un mot (subword) -> -100
            else:
                label_ids_aligned.append(-100)
            
            previous_word_idx = word_idx
        
        labels.append(label_ids_aligned)
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Application de la tokenization
print("Tokenization des datasets...")
tokenized_dataset = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset["train"].column_names
)

print("\nDatasets tokenizés :")
print(tokenized_dataset)
print("\nExemple tokenizé :")
print(f"input_ids : {tokenized_dataset['train'][0]['input_ids'][:20]}...")
print(f"labels    : {tokenized_dataset['train'][0]['labels'][:20]}...")

## 6. Chargement et configuration du modèle

### 6.1 Chargement de CamemBERT pré-entraîné

In [ ]:
print("Chargement du modèle CamemBERT...")
print("(Première exécution : téléchargement de ~440 MB)\n")

model = CamembertForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

# Déplacement vers GPU si disponible
model = model.to(device)

print(f"Modèle chargé sur {device}")
print(f"Nombre de paramètres : {sum(p.numel() for p in model.parameters()):,}")
print(f"Paramètres entraînables : {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

### 6.2 Data Collator

Le Data Collator gère le padding dynamique des batches pendant l'entraînement.

In [ ]:
data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    padding=True
)

print("Data Collator configuré (padding dynamique activé)")

## 7. Métriques d'évaluation

### 7.1 Fonction de calcul des métriques

Utilisation de seqeval pour respecter les contraintes BIO.

In [ ]:
def compute_metrics(eval_pred):
    """
    Calcule les métriques NER (precision, recall, F1) avec seqeval.
    
    Args:
        eval_pred: tuple (predictions, labels)
    
    Returns:
        dict avec precision, recall, f1, accuracy
    """
    predictions, labels = eval_pred
    
    # Predictions = logits -> argmax pour obtenir les classes
    predictions = np.argmax(predictions, axis=2)
    
    # Conversion en labels texte, en ignorant les -100
    true_labels = []
    pred_labels = []
    
    for prediction, label in zip(predictions, labels):
        true_labels_seq = []
        pred_labels_seq = []
        
        for pred_id, label_id in zip(prediction, label):
            if label_id != -100:  # Ignorer les tokens spéciaux
                true_labels_seq.append(id2label[label_id])
                pred_labels_seq.append(id2label[pred_id])
        
        true_labels.append(true_labels_seq)
        pred_labels.append(pred_labels_seq)
    
    # Calcul des métriques
    return {
        "precision": precision_score(true_labels, pred_labels),
        "recall": recall_score(true_labels, pred_labels),
        "f1": f1_score(true_labels, pred_labels)
    }

print("Fonction de métriques configurée")

## 8. Configuration de l'entraînement

### 8.1 Hyperparamètres

Configuration optimale pour le fine-tuning de CamemBERT (modèle français) sur NER.

In [ ]:
# Création du dossier de sortie
output_dir = BASE_DIR / "models" / "camembert_ner"
output_dir.mkdir(parents=True, exist_ok=True)

training_args = TrainingArguments(
    output_dir=str(output_dir),
    
    # Hyperparamètres d'entraînement
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    weight_decay=0.01,
    
    # Évaluation
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    
    # Logging
    logging_dir=str(output_dir / "logs"),
    logging_steps=50,
    
    # Optimisations
    fp16=torch.cuda.is_available(),  # Mixed precision si GPU
    dataloader_num_workers=4,
    
    # Autres
    seed=42,
    push_to_hub=False
)

print("Configuration d'entraînement :")
print(f"  Epochs           : {training_args.num_train_epochs}")
print(f"  Batch size train : {training_args.per_device_train_batch_size}")
print(f"  Learning rate    : {training_args.learning_rate}")
print(f"  FP16 (GPU)       : {training_args.fp16}")
print(f"  Output dir       : {training_args.output_dir}")

### 8.2 Initialisation du Trainer

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print("Trainer initialisé")
print(f"Nombre de steps d'entraînement : {len(tokenized_dataset['train']) // training_args.per_device_train_batch_size * training_args.num_train_epochs:,}")

## 9. Fine-tuning du modèle

### 9.1 Lancement de l'entraînement

Durée estimée : 
- Avec GPU : 30-60 minutes
- Avec CPU : 4-8 heures (non recommandé)

In [ ]:
print("=" * 70)
print("DÉBUT DU FINE-TUNING")
print("=" * 70)
print(f"Device : {device}")
print(f"Début : {time.strftime('%H:%M:%S')}")
print()

start_time = time.time()

# Entraînement
train_result = trainer.train()

# Temps total
total_time = time.time() - start_time
hours = int(total_time // 3600)
minutes = int((total_time % 3600) // 60)
seconds = int(total_time % 60)

print()
print("=" * 70)
print("FIN DE L'ENTRAÎNEMENT")
print("=" * 70)
print(f"Durée totale : {hours:02d}h {minutes:02d}m {seconds:02d}s")
print(f"Fin : {time.strftime('%H:%M:%S')}")

### 9.2 Métriques d'entraînement

In [ ]:
# Récupération des métriques
train_metrics = train_result.metrics

print("Métriques d'entraînement :")
print(f"  Training loss    : {train_metrics['train_loss']:.4f}")
print(f"  Training runtime : {train_metrics['train_runtime']:.2f}s")
print(f"  Samples/second   : {train_metrics['train_samples_per_second']:.2f}")

## 10. Évaluation sur le dev set

### 10.1 Prédictions et métriques

In [ ]:
print("=" * 70)
print("ÉVALUATION SUR DEV SET")
print("=" * 70)

# Évaluation
dev_results = trainer.evaluate()

print(f"\nMétriques globales (dev) :")
print(f"  Precision : {dev_results['eval_precision']:.4f}")
print(f"  Recall    : {dev_results['eval_recall']:.4f}")
print(f"  F1-score  : {dev_results['eval_f1']:.4f}")
print(f"  Loss      : {dev_results['eval_loss']:.4f}")

# Sauvegarde des résultats
dev_f1 = dev_results['eval_f1']

### 10.2 Rapport détaillé par classe

In [ ]:
# Prédictions détaillées
predictions_output = trainer.predict(tokenized_dataset["validation"])
predictions = np.argmax(predictions_output.predictions, axis=2)

# Conversion en labels
true_labels_dev = []
pred_labels_dev = []

for prediction, label in zip(predictions, predictions_output.label_ids):
    true_seq = []
    pred_seq = []
    for pred_id, label_id in zip(prediction, label):
        if label_id != -100:
            true_seq.append(id2label[label_id])
            pred_seq.append(id2label[pred_id])
    true_labels_dev.append(true_seq)
    pred_labels_dev.append(pred_seq)

# Rapport de classification
print("\n" + "=" * 70)
print("RAPPORT DE CLASSIFICATION DÉTAILLÉ (DEV)")
print("=" * 70)
print(classification_report(true_labels_dev, pred_labels_dev))

### 10.3 Analyse des meilleures et pires classes

In [ ]:
# Calcul du rapport par classe
report_dict = classification_report(true_labels_dev, pred_labels_dev, output_dict=True)

# Extraction des F1 par classe (hors moyennes)
classes_f1 = {
    k: v['f1-score'] 
    for k, v in report_dict.items() 
    if k not in ['micro avg', 'macro avg', 'weighted avg'] and k != 'O'
}

# Top 10 meilleures classes
top10_best = sorted(classes_f1.items(), key=lambda x: x[1], reverse=True)[:10]
print("Top 10 classes les mieux prédites :")
for label, f1_val in top10_best:
    support = report_dict[label]['support']
    print(f"  {label:35s} : F1={f1_val:.3f}  (support={support:4d})")

# Top 10 pires classes
top10_worst = sorted(classes_f1.items(), key=lambda x: x[1])[:10]
print("\nTop 10 classes les plus difficiles :")
for label, f1_val in top10_worst:
    support = report_dict[label]['support']
    print(f"  {label:35s} : F1={f1_val:.3f}  (support={support:4d})")

## 11. Évaluation sur le test set

### 11.1 Prédictions finales

In [ ]:
print("=" * 70)
print("ÉVALUATION FINALE SUR TEST SET")
print("=" * 70)

# Évaluation sur test
test_results = trainer.evaluate(tokenized_dataset["test"])

print(f"\nMétriques globales (test) :")
print(f"  Precision : {test_results['eval_precision']:.4f}")
print(f"  Recall    : {test_results['eval_recall']:.4f}")
print(f"  F1-score  : {test_results['eval_f1']:.4f}")
print(f"  Loss      : {test_results['eval_loss']:.4f}")

# Sauvegarde
test_f1 = test_results['eval_f1']

### 11.2 Comparaison Dev vs Test

In [ ]:
# Tableau comparatif
comparison_df = pd.DataFrame({
    'Split': ['Dev', 'Test'],
    'Precision': [dev_results['eval_precision'], test_results['eval_precision']],
    'Recall': [dev_results['eval_recall'], test_results['eval_recall']],
    'F1-score': [dev_results['eval_f1'], test_results['eval_f1']]
})

print("\n" + "=" * 70)
print("COMPARAISON DEV vs TEST")
print("=" * 70)
print(comparison_df.to_string(index=False))

# Visualisation
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(comparison_df))
width = 0.25

ax.bar(x - width, comparison_df['Precision'], width, label='Precision', color='#3498db')
ax.bar(x, comparison_df['Recall'], width, label='Recall', color='#e74c3c')
ax.bar(x + width, comparison_df['F1-score'], width, label='F1-score', color='#2ecc71')

ax.set_ylabel('Score', fontsize=12)
ax.set_title('Performance CamemBERT : Dev vs Test', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(comparison_df['Split'])
ax.legend()
ax.set_ylim(0, 1)
ax.grid(axis='y', alpha=0.3)

# Annotations
for i, (prec, rec, f1) in enumerate(zip(comparison_df['Precision'], comparison_df['Recall'], comparison_df['F1-score'])):
    ax.text(i - width, prec + 0.02, f'{prec:.3f}', ha='center', fontsize=9)
    ax.text(i, rec + 0.02, f'{rec:.3f}', ha='center', fontsize=9)
    ax.text(i + width, f1 + 0.02, f'{f1:.3f}', ha='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

## 12. Sauvegarde du modèle final

### 12.1 Sauvegarde complète

In [ ]:
# Sauvegarde du modèle et du tokenizer
final_model_dir = BASE_DIR / "models" / "camembert_ner_final"
final_model_dir.mkdir(parents=True, exist_ok=True)

print("Sauvegarde du modèle final...")
trainer.save_model(str(final_model_dir))
tokenizer.save_pretrained(str(final_model_dir))

print(f"Modèle sauvegardé dans : {final_model_dir}")

# Sauvegarde des métadonnées
metadata = {
    "model_name": "CamemBERT-base fine-tuned for NER",
    "model_checkpoint": model_checkpoint,
    "dataset": "MultiCoNER v2 (français)",
    "num_labels": num_labels,
    "num_train_samples": len(train_sentences),
    "num_dev_samples": len(dev_sentences),
    "num_test_samples": len(test_sentences),
    "training_args": {
        "epochs": training_args.num_train_epochs,
        "batch_size": training_args.per_device_train_batch_size,
        "learning_rate": training_args.learning_rate
    },
    "results": {
        "dev_precision": float(dev_results['eval_precision']),
        "dev_recall": float(dev_results['eval_recall']),
        "dev_f1": float(dev_results['eval_f1']),
        "test_precision": float(test_results['eval_precision']),
        "test_recall": float(test_results['eval_recall']),
        "test_f1": float(test_results['eval_f1'])
    },
    "date": time.strftime('%Y-%m-%d %H:%M:%S')
}

metadata_path = final_model_dir / "metadata.json"
with open(metadata_path, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

print(f"Métadonnées sauvegardées : {metadata_path}")

# Instructions pour télécharger depuis Colab
if IN_COLAB:
    print("\n" + "=" * 70)
    print("IMPORTANT - TÉLÉCHARGEMENT DU MODÈLE")
    print("=" * 70)
    print("Sur Colab, les fichiers dans /content sont temporaires.")
    print("Pour télécharger le modèle entraîné, exécutez cette cellule :")
    print()
    print("from google.colab import files")
    print("import shutil")
    print(f"shutil.make_archive('camembert_ner_final', 'zip', '{final_model_dir}')")
    print("files.download('camembert_ner_final.zip')")
    print("=" * 70)

### 12.2 Sauvegarde du mapping des labels

In [ ]:
# Sauvegarde des mappings
label_mapping = {
    "label2id": label2id,
    "id2label": id2label
}

label_mapping_path = final_model_dir / "label_mapping.json"
with open(label_mapping_path, 'w', encoding='utf-8') as f:
    json.dump(label_mapping, f, indent=2, ensure_ascii=False)

print(f"Mapping des labels sauvegardé : {label_mapping_path}")

### 12.3 Téléchargement du modèle (Colab uniquement)

⚠️ **IMPORTANT** : Sur Colab, exécutez cette cellule pour télécharger le modèle avant que la session expire !

In [ ]:
if IN_COLAB:
    print(" Création de l'archive du modèle...")
    
    from google.colab import files
    import shutil
    
    # Créer l'archive ZIP
    archive_name = 'camembert_ner_final'
    shutil.make_archive(archive_name, 'zip', final_model_dir)
    
    print(f"✓ Archive créée : {archive_name}.zip (~450 MB)")
    print(" Téléchargement automatique...")
    
    # Télécharger automatiquement
    files.download(f'{archive_name}.zip')
    
    print(" Modèle téléchargé avec succès !")
else:
    print("ℹ Cette cellule est uniquement pour Google Colab.")

## 13. Test de prédiction sur un exemple

### 13.1 Fonction de prédiction

In [ ]:
def predict_ner(text, model, tokenizer):
    """
    Prédit les entités nommées dans un texte.
    
    Args:
        text: str - texte à analyser
        model: modèle CamemBERT fine-tuné
        tokenizer: tokenizer CamemBERT
    
    Returns:
        list de tuples (token, label)
    """
    # Tokenization
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Prédiction
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Récupération des prédictions
    predictions = torch.argmax(outputs.logits, dim=2)
    
    # Décodage
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    predicted_labels = [id2label[p.item()] for p in predictions[0]]
    
    # Filtrage des tokens spéciaux
    results = []
    for token, label in zip(tokens, predicted_labels):
        if token not in [tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token]:
            results.append((token, label))
    
    return results

print("Fonction de prédiction définie")

### 13.2 Exemple de prédiction

In [ ]:
# Texte d'exemple
exemple_text = "La romancière américaine Susan Sontag est née à New York en 1933."

print("=" * 70)
print("EXEMPLE DE PRÉDICTION")
print("=" * 70)
print(f"\nTexte : {exemple_text}\n")

# Prédiction
predictions = predict_ner(exemple_text, model, tokenizer)

print("Prédictions :")
print(f"{'Token':<20} | {'Label':<30}")
print("-" * 52)
for token, label in predictions:
    # Mise en évidence des entités
    if label != 'O' and not label.startswith('domain'):
        print(f"{token:<20} | {label:<30} <<<")
    else:
        print(f"{token:<20} | {label:<30}")

## 14. Récapitulatif final

### 14.1 Résumé des performances

In [ ]:
print("=" * 70)
print("RÉCAPITULATIF FINAL - CAMEMBERT NER")
print("=" * 70)

print(f"\nModèle : {model_checkpoint}")
print(f"Dataset : MultiCoNER v2 (français)")
print(f"Nombre d'entités : {len(entity_labels)} types")
print(f"Nombre total de labels : {num_labels}")

print(f"\nDonnées d'entraînement :")
print(f"  Train : {len(train_sentences):6,} phrases")
print(f"  Dev   : {len(dev_sentences):6,} phrases")
print(f"  Test  : {len(test_sentences):6,} phrases")

print(f"\nHyperparamètres :")
print(f"  Epochs       : {training_args.num_train_epochs}")
print(f"  Batch size   : {training_args.per_device_train_batch_size}")
print(f"  Learning rate: {training_args.learning_rate}")

print(f"\nRésultats sur DEV :")
print(f"  Precision : {dev_results['eval_precision']:.4f}")
print(f"  Recall    : {dev_results['eval_recall']:.4f}")
print(f"  F1-score  : {dev_results['eval_f1']:.4f}")

print(f"\nRésultats sur TEST :")
print(f"  Precision : {test_results['eval_precision']:.4f}")
print(f"  Recall    : {test_results['eval_recall']:.4f}")
print(f"  F1-score  : {test_results['eval_f1']:.4f}")

print(f"\nModèle sauvegardé : {final_model_dir}")
print("=" * 70)

## 15. Conclusion

### Points clés

**Architecture :**
- CamemBERT-base (110M paramètres) pré-entraîné sur 138 GB de texte français
- Fine-tuné sur 16,548 phrases annotées du MultiCoNER v2
- Tokenization subword avec alignement automatique des labels

**Performance :**
- Le modèle Transformer surpasse généralement les approches CRF classiques de 10-20% en F1
- Capture le contexte bidirectionnel profond
- Gère mieux les entités complexes et ambiguës

**Utilisation :**
- Le modèle est prêt pour la production
- Peut être chargé avec `from_pretrained(model_dir)`
- Compatible avec l'API Hugging Face

### Prochaines étapes possibles

1. **Optimisation** : Recherche d'hyperparamètres, plus d'epochs
2. **Ensembles** : Combiner CRF + CamemBERT
3. **Augmentation** : Techniques d'augmentation de données
4. **Déploiement** : API FastAPI, conteneurisation Docker
5. **Analyse** : Étude approfondie des erreurs, amélioration ciblée